In [ ]:
from netgen.geom2d import unit_square
from ngsolve import *
import netgen.gui
import matplotlib.pyplot as plt

# Example 10.1a

a) Implement the weak formulation of the Dirichlet boundary conditions on the unit square with non homogenous Dirichlet boundary conditions uD = y(1 - y)(1 - x) and a zero right hand side. In NGSolve use the spaces:

```
V = H1(mesh, order = k)
Q = SurfaceL2(mesh, order = k-1)
```

\begin{align}
-\Delta u &= f &on\ \Omega=[0,1]^2 \\
u &= u_D &on\ \Gamma_D = \partial \Omega
\end{align}
with $f = 0$ and $u_D = y(1 - y)(1 - x)$

In [ ]:
# Mesh
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

mesh.GetBoundaries()

In [ ]:
### SETUP parameters for the problem as given in the exercise sheet ###
#######################################################################
dirichlet = "left|bottom|right|top"
rhs = Parameter(0.0)
# Weak Dirichlet boundary condition
ud = y*(1-y)*(1-x) # CoefficientFunction

# FES(es)
k = 2
V = H1(mesh,order = k) #, dirichlet = dirichlet)
Q = SurfaceL2(mesh, order = k-1)
fes = FESpace([V, Q])
fes
(u, p), (v, q) = fes.TnT()

# Bilinear Form
a = BilinearForm(fes) #,symmetric=True)
a += grad(u)*grad(v)*dx
a += u*q*ds(definedon=dirichlet)
a += p*v*ds(definedon=dirichlet)

# Setup solution GridFunction
# ud = 0 if bnd != left
gf = GridFunction(fes) # solution
gfu = gf.components[0]
gfu.Set(ud) # set boundary condition
Draw(gfu, mesh, 'u_D')

# Linear Form
f = LinearForm(fes)
f += rhs*v*dx + ud*q*ds(definedon=dirichlet)

In [ ]:
with TaskManager():
    a.Assemble()
    f.Assemble()
    # Homogenize
    #r = f.vec.CreateVector()
    #r.data = f.vec - a.mat * gf.vec
    # Solve
    #gf.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
    solvers.BVP(bf=a, lf=f, gf=gf)


Draw(gfu, mesh, 'u_a')
Draw(-grad(gfu), mesh, 'grad(u_a)')

# Example 10.1b

b) Follow the same steps as in the derivation of the mixed methods for the weak formulation of the Dirichlet boundary conditions in the lecture to derive a mixed method for the Poisson problem with Robin boundary conditions

\begin{align}
-\Delta u &= f &on\ \Omega=[0,1]^2 \\
u - u_D &= \epsilon \frac{\partial u}{\partial n} &on\ \Gamma_R = \partial \Omega
\end{align}

with a given $\epsilon$ and $u_D$. Implement this method with $f = 100 e^{-100 ((x-0.5)^2+(y-0.5)^2)}$.

In [ ]:
### SETUP parameters for the problem as given in the exercise sheet ###
#######################################################################
robin = "left|bottom|right|top"

# The exponent gives the squared distance to (0.5,0.5) in the 2D plane --> circle/sphere around middle point
sphere = (x-0.5)*(x-0.5)+(y-0.5)*(y-0.5)
# The resulting function can be interpreted as a Gaußian Distribution around the "spheres" middle point (0.5,0.5)
rhs = 100*exp(-100*sphere)
Draw(sphere, mesh, 'sphere')
Draw(rhs, mesh, 'rhs')
print(rhs)

eps = Parameter(1.0)
ud = Parameter(1.0)

# FES(es)
k = 2
V = H1(mesh,order = k) #, dirichlet = dirichlet)
Q = SurfaceL2(mesh, order = k-1)
fes = FESpace([V, Q])
fes
(u, p), (v, q) = fes.TnT()

# Bilinear Form
a = BilinearForm(fes) #,symmetric=True)
a += grad(u)*grad(v)*dx
a += p*v*ds(definedon=robin)
a += u*q*ds(definedon=robin)
a += eps*p*q*ds(definedon=robin)

# Linear Form
f = LinearForm(fes)
f += rhs*v*dx + ud*q*ds(definedon=robin)

# Setup solution GridFunction
gf = GridFunction(fes) # solution
gfu = gf.components[0] # Compound Space --> We want solution of u later


In [ ]:
with TaskManager():
    a.Assemble()
    f.Assemble()
    # Homogenize
    #r = f.vec.CreateVector()
    #r.data = f.vec - a.mat * gf.vec
    # Solve
    #gf.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
    solvers.BVP(bf=a, lf=f, gf=gf)


Draw(gfu, mesh, 'u_b')
Draw(-grad(gfu), mesh, 'grad(u_b)')